In [53]:
import boto
import dask.dataframe as dd
import pandas as pd
import os
import re
import logging
from dask.delayed import delayed
from distributed import Client, get_worker
from functools import partial

In [32]:
s3 = boto.connect_s3(profile_name='peeriq')
bucket = s3.get_bucket('peeriq-etl')

In [15]:
client = Client('10.0.2.76:8786')

distributed.comm.tcp - DEBUG - Setting TCP keepalive: idle=10, interval=2
distributed.comm.tcp - DEBUG - Setting TCP keepalive: idle=10, interval=2
distributed.client - DEBUG - Started scheduling coroutines. Synchronized


In [49]:
peeriq_pattern = re.compile('etl/Input/Public/LC/Monthly/LoanBorrower/LC_LoanBorrower_2017-03-31.*\.csv')
prefix = 'etl/Input/Public/LC/Monthly/LoanBorrower/'

In [ ]:
pattern = re.compile('\d{8}/loan_stats.csv')

In [50]:
host = f's3://{bucket.name}'
urls = [key.generate_url(expires_in=600) for key in bucket.list(prefix=prefix) if peeriq_pattern.search(key.name)]
print(f'found {len(urls)} urls')

found 529 urls


In [54]:
def read_csv_async(url):
    df = pd.read_csv(url, index_col=False)
    worker = get_worker()
    df['worker'] = worker.address
    return df

In [56]:
# create a list of 'future' data frame objects
future_frames = [delayed(read_csv_async)(url) for url in urls]
# create a 'future' DataFrame object
df_future = dd.from_delayed(future_frames)
cols = ['loan_month', 'originator_loan_grade', 'outstanding_principal_eop']
df_by_worker_future = df_future[cols].groupby(['loan_month', 'originator_loan_grade']).sum()

distributed.client - DEBUG - Waiting on futures to clear before gather
distributed.client - DEBUG - Client receives message {'op': 'key-in-memory', 'key': 'read_csv_async-ebe637c0-afa2-4a49-85a6-257a8e497c33', 'type': b'\x80\x04\x95#\x00\x00\x00\x00\x00\x00\x00\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94.'}
distributed.client - DEBUG - Release key read_csv_async-ebe637c0-afa2-4a49-85a6-257a8e497c33


In [57]:
%%time
df = df_by_worker_future.compute()

distributed.client - DEBUG - Waiting on futures to clear before gather
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - WARNING - Client report stream closed to scheduler
distributed.client - INFO - Reconnecting...
distributed.client - INFO - Reconnecting...
distributed.client - DEBUG - Release key ('dataframe-groupby-sum-agg-b052cbe1e6edbe7480b0c060103cbabd', 0)
distributed.utils - ERROR - ('dataframe-groupby-sum-agg-b052cbe1e6edbe7480b0c060103cbabd', 0)
Traceback (most recent call last):
  File "C:\Users\Andrew Meier\Miniconda2\envs\py3\lib\site-packages\distributed\client.py", line 986, in _gather
    st = self.futures[key]
KeyError: "('dataframe-groupby-sum-agg-b052cbe1e6edbe7480b0c060103cbabd', 0)"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Andrew Meier\Miniconda2\envs\py3\lib\site-packages\distributed\utils.py", line 212, in f
    result[0] = yield make_coro(

CancelledError: ('dataframe-groupby-sum-agg-b052cbe1e6edbe7480b0c060103cbabd', 0)

In [59]:
df.head()

,loan_amnt
grade,
A,2.080676e+09
B,3.472829e+09
C,3.556909e+09
D,2.157448e+09
E,1.272601e+09
